# DUMMY

* https://github.com/unslothai/unsloth
* https://docs.unsloth.ai/get-started/installing-+-updating/windows-installation  METHOD#2 (CC,CXX, CL path)

* conda deactivate
* conda remove --name env_llm_qlora --all -y

Install cudatoolkit
* conda create --name env_llm_qlora python=3.21 pytorch-cuda=12.1 pytorch cudatoolkit -c pytorch -c nvidia -y
* conda activate env_llm_qlora


* ####python -m pip install xformers | conda install -c conda-forge xformers |

* python -m pip install -U xformers --index-url https://download.pytorch.org/whl/cu121
* [Blackwell] python -m pip install -U xformers --index-url https://download.pytorch.org/whl/nightly/cu128 (python -m pip uninstall xformers -y)

python -m pip install "unsloth[windows] @ git+https://github.com/unslothai/unsloth.git"
python -m pip install --no-deps trl peft accelerate bitsandbytes
python -m pip install ipywidgets ipykernel
python -m pip install jmespath
python -m pip install --upgrade accelerate
python -m pip install tensorboard

[Blackwell]
python -m pip uninstall torch torchvision torchaudio -y
python -m pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu128
python -m pip install --no-deps xformers==0.0.29.post3

Patch: num_proc = 1

# Imports

In [1]:
import os
import torch
import json
import random
# Unsloth always first as import, otherwise will create some inconsistent errors
from unsloth import FastLanguageModel, is_bfloat16_supported
from unsloth.chat_templates import get_chat_template, train_on_responses_only

from trl import SFTTrainer
from datasets import load_dataset, Dataset
from threading import Thread
from transformers import TrainingArguments, DataCollatorForSeq2Seq, TextIteratorStreamer, TextStreamer

from pathlib import Path

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
cuda_available = torch.cuda.is_available()
print(f"CUDA disponible: {cuda_available}")

# Mostrar el dispositivo actual
if cuda_available:
    device = torch.device("cuda")
    print(f"Dispositivo actual: {torch.cuda.get_device_name(0)}")
    print(f"Cantidad de GPUs disponibles: {torch.cuda.device_count()}")
else:
    device = torch.device("cpu")
    print("Se está utilizando la CPU.")

CUDA disponible: True
Dispositivo actual: NVIDIA GeForce RTX 5070 Ti
Cantidad de GPUs disponibles: 2


In [3]:
max_seq_length = 2048                                                     # Máxima longitud de secuencia (ajústala si necesitas más contexto)
dtype = None                 # Puede ser especificado, en este ejemplo lo dejamos por defecto
load_in_4bit = True          # Usamos 4bit para optimizar uso de VRAM
SEED        = 4242
DATA_PATH_TRAIN   = Path("datasets/curated/interview_questions/dataset_train.jsonl")
DATA_PATH_VAL   = Path("datasets/curated/interview_questions/dataset_val.jsonl")
MODEL_NAME  = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"

OUTPUT_DIR  = Path("models/master_model")
SYSTEM_PROMPT = {
    "role": "system",
    "content": """You are an expert assistant specialized in teaching professional English to non-native speakers. You have an advanced command of grammar, business communication, and technical topics such as artificial intelligence, programming, cloud computing, DevOps, data engineering, and project management. You also know how to hold helpful, educational, and realistic conversations simulating real-life workplace situations.

Your task is to help the user improve their professional English by always responding in a **strict XML format** that includes grammar corrections, brief technical clarifications, and a natural conversation follow-up.

---

**🧠 MANDATORY RESPONSE FORMAT (no variations)**  
Every response must follow this **exact XML structure**. Do not modify or skip any of the following tags:

<response>
<enhancement>Grammar correction written clearly and naturally.</enhancement>
<expert-answer>Brief technical clarification or improvement with a natural example.</expert-answer>
<continue>Conversational follow-up in fluent, natural English to keep the dialogue going.</continue>
</response>

**Do NOT add any titles, Markdown, bullet points, line breaks, or comments outside the XML tags. Only return what’s inside the <response> block.**

---

**📚 GUIDELINES FOR EACH SECTION:**

1. <enhancement>  
Fix the grammar mistake in the user’s sentence. Keep it short, natural, and clearly improved.  
If only one word needs fixing, don’t rewrite the entire sentence unnecessarily.  
Use vocabulary and phrasing typical of real workplace communication.

2. <expert-answer>  
Provide a short technical correction or explanation if the user’s input has a conceptual error.  
**Always include a short example** of how a native professional would say it.  
If there’s no technical error, you may provide a useful improvement instead.

3. <continue>  
Continue the conversation naturally by asking a **relevant follow-up question** or giving a brief, engaging comment.  
Do not repeat the correction. Stay professional, friendly, and aligned with the topic.

---

**✅ STYLE AND TONE:**

- Never begin with "Sure!", "Of course," or other filler phrases.
- Keep responses concise, clear, and suitable for text-to-speech use.
- Avoid long or academic explanations.
- Focus on one relevant mistake per interaction (e.g., verb tense, preposition, article, countability, vocabulary choice, etc.).
- Always maintain an empathetic and supportive tone, like a tutor helping a learner prepare for technical interviews or workplace conversations.

---

**🎯 MAIN OBJECTIVE:**

Your role is to help the user:

- Improve their grammar and fluency in a professional/technical context.
- Learn from realistic English errors made by intermediate-level speakers.
- Practice how to speak effectively in job interviews, meetings, emails, and tech discussions.
- Get practical examples of correct English phrasing used by fluent professionals.

---

**🔁 COMPLETE EXAMPLE OF A VALID RESPONSE:**

User input:  
"I finish the deploy yesterday night but server was not response."

Your response must be exactly:

<response><enhancement>I finished the deployment last night, but the server didn't respond.</enhancement><expert-answer>'Deployment' is the correct noun, and past tense is needed here. For example: 'We deployed the update on Friday.'</expert-answer><continue>Do you remember what error or message the server showed?</continue></response>

---

From now on, **always respond in this XML format**. Never deviate from this structure. No headings, no lists, no extra formatting. Just a clear, accurate, and friendly learning response for each user message.

---"""
}

| Módulo           | ¿Cuándo incluirlo en `target_modules`?                                             | Notas adicionales |
|------------------|------------------------------------------------------------------------------------|-------------------|
| `q_proj`         | ✅ **Siempre**                                                                     | La matriz Query determina cómo se consulta la atención, esencial para aprendizaje de contextos específicos y conversaciones. |
| `k_proj`         | ✅ **Siempre**                                                                     | La matriz Key determina cómo se organizan las representaciones internas del contexto. Necesaria para capturar patrones semánticos. |
| `v_proj`         | ✅ **Siempre**                                                                     | La matriz Value representa el valor real o contenido semántico; fundamental para generar respuestas coherentes y precisas. |
| `o_proj`         | ✅ **Siempre o casi siempre**                                                       | Proyección de salida final en atención; afecta directamente la calidad y claridad de las respuestas. Suele mejorar la coherencia. |
| `gate_proj`      | 🔶 **A veces** (Útil especialmente en tareas complejas o textos largos)            | Controla la activación en capas FFN; ayuda en tareas con mucha información o patrones complejos. Puede mejorar calidad, pero añade memoria y tiempo. |
| `up_proj`        | 🔶 **A veces** (Útil en datasets grandes o muy diversos)                           | Expande la dimensión interna en FFN. Agregarla mejora ligeramente capacidad expresiva pero consume más memoria. |
| `down_proj`      | 🔶 **A veces** (Útil junto con `up_proj` en tareas complejas o datasets variados)  | Reduce la dimensión en FFN, actúa junto con `up_proj`. Se recomienda si se usa `up_proj`. |

### 📌 **Recomendaciones prácticas rápidas:**

- **Casos estándar (p. ej. chatbots, asistentes):**
  ```python
  target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"]
  ```

- **Casos avanzados (textos largos, tareas complejas, fine-tuning intensivo):**
  ```python
  target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
  ```

In [4]:
from typing import List, Dict

def _normalize_msg(msg: Dict) -> Dict:
    """Devuelve un mensaje válido (role/content) y fuerza content a str."""
    role = msg.get("role", "")
    content = msg.get("content", "")
    if not isinstance(content, str):
        content = json.dumps(content, ensure_ascii=False)
    return {"role": role, "content": content}

def safe_load_jsonl(path: Path, *, remove_metadata: bool = False) -> List[Dict]:
    """
    • Lee un JSONL estilo ShareGPT.
    • Inyecta SYSTEM_PROMPT.
    • Opcionalmente elimina mensajes con role == "metadata".
    • Convierte content no-str a str para evitar errores posteriores.
    """
    clean_rows, bad_rows = [], 0

    with path.open(encoding="utf-8") as f:
        for idx, line in enumerate(f, 1):
            line = line.strip()
            if not line:
                continue
            try:
                convo = json.loads(line)
            except json.JSONDecodeError:
                bad_rows += 1
                continue

            # debe ser lista de dicts role/content
            if not (
                isinstance(convo, list)
                and all(isinstance(m, dict) for m in convo)
                and all("role" in m and "content" in m for m in convo)
            ):
                bad_rows += 1
                continue

            # --- FILTRO metadata + normalización ---
            filtered = []
            for m in convo:
                if remove_metadata and m["role"] == "metadata":
                    continue
                filtered.append(_normalize_msg(m))

            # salta conversaciones vacías (si quitamos todo)
            if not filtered:
                bad_rows += 1
                continue

            full_convo = [SYSTEM_PROMPT.copy()] + filtered
            clean_rows.append({"conversation": full_convo})

    if bad_rows:
        print(f"⚠️  Se descartaron {bad_rows} líneas defectuosas o vacías.")
    print(f"✅ Conversaciones válidas: {len(clean_rows)}")
    return clean_rows

In [5]:
# ───────────────────────── Preparación modelo/tokenizer ─────────────────────
def get_model_and_tokenizer():
    model, tok = FastLanguageModel.from_pretrained(
        model_name      = MODEL_NAME,
        max_seq_length  = max_seq_length,
        load_in_4bit    = load_in_4bit,
    )
    model = FastLanguageModel.get_peft_model(
        model,
        r               = 32,
        lora_alpha      = 16,
        lora_dropout    = 0.05,
        target_modules  = ["q_proj","k_proj","v_proj","o_proj"],
        use_gradient_checkpointing = "unsloth",
        random_state    = SEED,
    )
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token
    tok = get_chat_template(tok, chat_template="llama-3.2")
    return model, tok

In [6]:
model, tokenizer = get_model_and_tokenizer()

==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 5070 Ti. Num GPUs = 2. Max memory: 15.92 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0.dev20250513+cu128. CUDA: 12.0. CUDA Toolkit: 12.8. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31+8fc8ec5a.d20250511. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.4.7 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [7]:
# Si el tokenizador no tiene definido un token de padding, lo asignamos a uno de los tokens reservados.
if tokenizer.pad_token is None:
    # Elegimos usar <|reserved_special_token_0|> como token de padding
    tokenizer.pad_token = "<|reserved_special_token_0|>"
    print("Se asignó <|reserved_special_token_0|> como token PAD.")

# Verificamos que el modelo tenga configurado el pad_token_id.
if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.pad_token_id
    print(f"Se configuró el pad_token_id del modelo a: {model.config.pad_token_id}")

# Asegurarse de que el token EOS esté configurado (por lo general es <|end_of_text|>)
if not hasattr(tokenizer, "eos_token") or tokenizer.eos_token is None:
    # En muchos modelos, el EOS es "<|end_of_text|>"; ajusta según tu modelo.
    tokenizer.eos_token = "<|end_of_text|>"
    print("Se asignó <|end_of_text|> como token EOS.")

print("Configuración de tokens especiales completada.")

Configuración de tokens especiales completada.


In [8]:
convs_train = safe_load_jsonl(DATA_PATH_TRAIN, remove_metadata=True)            # lista de dicts {"conversation": [...]}

texts_train = [
    tokenizer.apply_chat_template(
        item["conversation"],
        tokenize=False,
        add_generation_prompt=False
    )
    for item in convs_train
]

dataset_train = Dataset.from_dict({"text": texts_train})  # ← siempre string → nunca ArrowInvalid
# Imprimir un ejemplo del dataset original para validación
print("\nEjemplo de conversación original (sin formatear):")
print(texts_train[0])

convs_val = safe_load_jsonl(DATA_PATH_VAL, remove_metadata=True)            # lista de dicts {"conversation": [...]}

# 2) convertir cada conversación a string con la plantilla
texts_val = [
    tokenizer.apply_chat_template(
        item["conversation"],
        tokenize=False,
        add_generation_prompt=False
    )
    for item in convs_val
]

dataset_val = Dataset.from_dict({"text": texts_val})  # ← siempre string → nunca ArrowInvalid

✅ Conversaciones válidas: 1192

Ejemplo de conversación original (sin formatear):
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

You are an expert assistant specialized in teaching professional English to non-native speakers. You have an advanced command of grammar, business communication, and technical topics such as artificial intelligence, programming, cloud computing, DevOps, data engineering, and project management. You also know how to hold helpful, educational, and realistic conversations simulating real-life workplace situations.

Your task is to help the user improve their professional English by always responding in a **strict XML format** that includes grammar corrections, brief technical clarifications, and a natural conversation follow-up.

---

**🧠 MANDATORY RESPONSE FORMAT (no variations)**  
Every response must follow this **exact XML structure**. Do not modify or skip any of the following tag

We verify masking is actually done:

In [9]:
dataset_train[0]

{'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou are an expert assistant specialized in teaching professional English to non-native speakers. You have an advanced command of grammar, business communication, and technical topics such as artificial intelligence, programming, cloud computing, DevOps, data engineering, and project management. You also know how to hold helpful, educational, and realistic conversations simulating real-life workplace situations.\n\nYour task is to help the user improve their professional English by always responding in a **strict XML format** that includes grammar corrections, brief technical clarifications, and a natural conversation follow-up.\n\n---\n\n**🧠 MANDATORY RESPONSE FORMAT (no variations)**  \nEvery response must follow this **exact XML structure**. Do not modify or skip any of the following tags:\n\n<response>\n<enhancement>Grammar correction written cl

In [10]:
training_args = TrainingArguments(
    output_dir                  = str(OUTPUT_DIR),
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    learning_rate               = 2e-5,
    num_train_epochs            = 3,
    warmup_steps                = 10,
    logging_steps               = 10,
    save_total_limit            = 2,
    optim                       = "adamw_8bit",
    fp16                        = not is_bfloat16_supported(),
    bf16                        = is_bfloat16_supported(),
    seed                        = SEED,
    report_to                   = "tensorboard",
    eval_steps                  = 50,
    save_strategy               = "steps",
    save_steps                  = 100,
)
print("→ TrainingArguments configurados.")

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer)

# 7) Crear SFTTrainer
print("🔄 Configurando SFTTrainer…")
trainer = SFTTrainer(
    model               = model,
    tokenizer           = tokenizer,
    train_dataset       = dataset_train,
    eval_dataset        = dataset_val,
    dataset_text_field  = "text",
    max_seq_length      = max_seq_length,
    data_collator       = data_collator,
    packing             = False,
    args                = training_args,
)
# 8) Enmascarar solo <assistant> para la pérdida
# IF: ZeroDivisionError raises, you have to check your dataset, probable exceeded the max sq lenth
print("🔄 Aplicando train_on_responses_only…")
trainer = train_on_responses_only(
    trainer,
    instruction_part="<|start_header_id|>system<|end_header_id|>\n\n",
    response_part="<|start_header_id|>assistant<|end_header_id|>\n\n",
)


→ TrainingArguments configurados.
🔄 Configurando SFTTrainer…


Unsloth: Tokenizing ["text"] (num_proc=24):   0%|          | 0/1192 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=24):   0%|          | 0/149 [00:00<?, ? examples/s]

🔄 Aplicando train_on_responses_only…


Map (num_proc=24):   0%|          | 0/1192 [00:00<?, ? examples/s]

Map (num_proc=24):   0%|          | 0/149 [00:00<?, ? examples/s]

In [15]:
tokenizer.decode(trainer.train_dataset[0]["input_ids"])

'<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou are an expert assistant specialized in teaching professional English to non-native speakers. You have an advanced command of grammar, business communication, and technical topics such as artificial intelligence, programming, cloud computing, DevOps, data engineering, and project management. You also know how to hold helpful, educational, and realistic conversations simulating real-life workplace situations.\n\nYour task is to help the user improve their professional English by always responding in a **strict XML format** that includes grammar corrections, brief technical clarifications, and a natural conversation follow-up.\n\n---\n\n**🧠 MANDATORY RESPONSE FORMAT (no variations)**  \nEvery response must follow this **exact XML structure**. Do not modify or skip any of the following tags:\n\n<response>\n<enhancement>Grammar correction wr

In [17]:
# You should not see the system or user part
space = tokenizer("X", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX<response><enhancement>Consistency ensures that all nodes in the system reflect the same data at the same time, which is critical for maintaining data integrity.</enhancement><expert-answer>Well done! I woul

In [18]:
print("▶️ Iniciando entrenamiento…")
trainer.train()

▶️ Iniciando entrenamiento…


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,192 | Num Epochs = 2 | Total steps = 148
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 27,262,976/8,000,000,000 (0.34% trained)


Step,Training Loss
10,0.619200
20,0.623900
30,0.558600
40,0.563700
50,0.587100
60,0.596500
70,0.591200
80,0.593500
90,0.529900
100,0.561600


TrainOutput(global_step=148, training_loss=0.5746716422003668, metrics={'train_runtime': 1414.0605, 'train_samples_per_second': 1.686, 'train_steps_per_second': 0.105, 'total_flos': 1.0045094168376115e+17, 'train_loss': 0.5746716422003668})

tensorboard --logdir models/master_model/runs/May31_17-50-27_magod --port 6006

In [19]:
# Definimos la ruta para guardar el modelo fine-tuneado
output_model_dir = str(OUTPUT_DIR)
os.makedirs(output_model_dir, exist_ok=True)

print("Guardando el modelo y el tokenizador en:", output_model_dir)
model.save_pretrained(output_model_dir)
tokenizer.save_pretrained(output_model_dir)
print("Modelo y tokenizador guardados correctamente.")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

Guardando el modelo y el tokenizador en: models/master_model
Modelo y tokenizador guardados correctamente.


In [11]:
if True:
    from unsloth import FastLanguageModel
    from unsloth.chat_templates import get_chat_template
    import json, torch

    # ───────────────────────────────────────────────────────
    # CONFIGURATION FLAG:
    # ───────────────────────────────────────────────────────
    # Set this to False if you do NOT want to load/run the original model
    # (to save GPU memory). Only the fine-tuned model will be used.
    USE_ORIGINAL = False

    # ───────────────────────────────────────────────────────
    # 1) Cargar modelos
    # ───────────────────────────────────────────────────────
    # Always load the fine-tuned model (model_ft).
    model_ft, tok_ft = FastLanguageModel.from_pretrained(
        str(OUTPUT_DIR),
        max_seq_length=max_seq_length,
        dtype=dtype,
        load_in_4bit=load_in_4bit,
    )
    FastLanguageModel.for_inference(model_ft)

    # Condicional: solo cargar el modelo original si USE_ORIGINAL es True.
    if USE_ORIGINAL:
        model_orig, tok_orig = FastLanguageModel.from_pretrained(
            MODEL_NAME,
            max_seq_length=max_seq_length,
            dtype=dtype,
            load_in_4bit=load_in_4bit,
        )
        FastLanguageModel.for_inference(model_orig)
    else:
        model_orig = None
        tok_orig = None

    # ───────────────────────────────────────────────────────
    # 2) Seleccionar y mover modelos al mismo device
    # ───────────────────────────────────────────────────────
    # Decidir explícitamente qué GPU/CPU queremos usar.
    # Si tiene varias GPUs, cámbielo por "cuda:0" o "cuda:1" según su preferencia.
    # Aquí forzamos cuda:0 si hay GPU disponible, de lo contrario CPU.
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("→ using device:", device)

    # Mover siempre el modelo fine-tuned
    model_ft = model_ft.to(device)

    # Si cargamos el modelo original, muévalo al mismo device:
    if USE_ORIGINAL:
        model_orig = model_orig.to(device)

    # Aplicar las plantillas de chat a los tokenizadores
    tok_ft = get_chat_template(tok_ft, chat_template="llama-3.2")
    tok_ft.pad_token = tok_ft.eos_token

    if USE_ORIGINAL:
        tok_orig = get_chat_template(tok_orig, chat_template="llama-3.2")
        tok_orig.pad_token = tok_orig.eos_token

    # ───────────────────────────────────────────────────────
    # 3) Función de generación segura (soporta tensor-o-dict)
    # ───────────────────────────────────────────────────────
    def generate_response(model, tokenizer, messages):
        """
        model: un FastLanguageModel (ya trasladado a `device`)
        tokenizer: el template ya ajustado (pad_token = eos_token)
        messages: lista de dicts con roles y contenidos
        """
        # A) Codificar el prompt completo con apply_chat_template:
        enc = tokenizer.apply_chat_template(
            messages, tokenize=True, add_generation_prompt=True, return_tensors="pt"
        )

        # B) Asegurar que input_ids (y máscaras) estén en el mismo device que el modelo
        if isinstance(enc, torch.Tensor):
            enc = {"input_ids": enc.to(device)}
        else:
            enc = {k: v.to(device) for k, v in enc.items()}

        # C) Ejecutar la llamada a model.generate(...) sobre ese mismo device
        out = model.generate(
            **enc,
            max_new_tokens=256,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.2,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
        )

        # D) Extraer solo los tokens recién generados, dejando fuera el prompt original
        new_toks = out[0, enc["input_ids"].shape[-1]:]

        # E) Decodificar y devolver string final
        return tokenizer.decode(
            new_toks,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
        )

    # ───────────────────────────────────────────────────────
    # 4) Comparativa con el set de validación
    # ───────────────────────────────────────────────────────
    with open(DATA_PATH_VAL, encoding="utf-8") as fh:
        for idx, line in enumerate(fh, 1):
            convo = json.loads(line.strip())
            last_user = next(
                (m["content"] for m in reversed(convo) if m.get("role") == "user"),
                None
            )
            if not last_user:
                continue

            # Construir el prompt: sistema + usuario
            prompt = [
                SYSTEM_PROMPT,
                {"role": "user", "content": last_user}
            ]

            print(f"\n=== Ejemplo #{idx} ===")
            print("🗣️ Usuario :", last_user)

            # 1) Generar con el fine-tuned model
            respuesta_ft = generate_response(model_ft, tok_ft, prompt)
            print("🔸 Fine-tuned:", respuesta_ft)

            # 2) Si está habilitado, generar también con el modelo original
            if USE_ORIGINAL:
                respuesta_orig = generate_response(model_orig, tok_orig, prompt)
                print("🔸 Original  :", respuesta_orig)
            else:
                # Si NO estamos usando el modelo original, opcionalmente indicamos que se omite:
                print("🔸 Original  : [omitted — flag USE_ORIGINAL=False]")

            print("=" * 60)

            if idx >= 10:
                break


==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 5070 Ti. Num GPUs = 2. Max memory: 15.92 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0.dev20250513+cu128. CUDA: 12.0. CUDA Toolkit: 12.8. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31+8fc8ec5a.d20250511. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


→ using device: cuda:0

=== Ejemplo #1 ===
🗣️ Usuario : Fog Computing make security better by local data process and less sensitive data sent to cloud, less risk to attacks.
🔸 Fine-tuned: <response><enhancement>Fog Computing enhances security through localized processing and reduced transmission of sensitive information to the Cloud, thereby minimizing attack risks.</enhancement><expert-answer>You’re close! The exact wording is ‘Fog Computing enhances security through localized processing and reduced transmission of sensitive information to the Cloud, thereby minimizing attack risks.’ This reduces latency and exposure while keeping critical data locally secure.</expert-answer><continue>Have you implemented Fog Computing solutions in your projects yet? How do they compare to traditional approaches?</continue></response>
🔸 Original  : [omitted — flag USE_ORIGINAL=False]

=== Ejemplo #2 ===
🗣️ Usuario : Real-time data is important for monitor health of equipment and give insights for main